In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
pd.options.display.max_columns=100
pd.options.display.max_rows=100
from encoding import FreqeuncyEncoding
from custom_estimator_regressor import Estimator
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from sklearn.externals import joblib
import datetime

In [10]:
import numpy
numpy.__version__

'1.16.2'

In [2]:
oof1=pd.read_csv('OOF_lgb_118_seed_100.csv')
oof2=pd.read_csv('OOF_lgb_118_seed_200.csv')
oof4=pd.read_csv('OOF_lgb_946_seed_100.csv')
oof5=pd.read_csv('OOF_lgb_946_seed_200.csv')
oof7=pd.read_csv('OOF_catboost_118_lr_0.02_all_cat_FE.csv')
oof8=pd.read_csv('OOF_catboost_118_lr_0.02_all_cat_FE_seed_200.csv')

In [3]:
preds1=pd.read_csv('preds_lgb_118_seed_100.csv')
preds2=pd.read_csv('preds_lgb_118_seed_200.csv')
preds4=pd.read_csv('preds_lgb_946_seed_100.csv')
preds5=pd.read_csv('preds_lgb_946_seed_200.csv')
preds7=pd.read_csv('catboost_118_lr_0.02_all_cat_FE.csv')
preds8=pd.read_csv('catboost_118_lr_0.02_all_cat_FE_seed_200.csv')

In [4]:
meta_train=pd.DataFrame({'lgb_118_100':oof1.iloc[:,0],'lgb_118_200':oof2.iloc[:,0],
             'lgb_946_100':oof4.iloc[:,0],'lgb_946_200':oof5.iloc[:,0],
             'catb_118_100':oof7.iloc[:,0],
             'catb_118_200':oof8.iloc[:,0]
                        })
target=pd.read_csv('train.csv',usecols=['amount_spent_per_room_night_scaled']).amount_spent_per_room_night_scaled

In [5]:
meta_test=pd.DataFrame({'lgb_118_100':preds1.iloc[:,0],'lgb_118_200':preds2.iloc[:,0],
             'lgb_946_100':preds4.iloc[:,0],'lgb_946_200':preds5.iloc[:,0],
             'catb_118_100':preds7.iloc[:,0],
            'catb_118_200':preds8.iloc[:,0]
                       }
                      )

In [6]:
meta_test.head()

,catb_118_100,catb_118_200,lgb_118_100,lgb_118_200,lgb_946_100,lgb_946_200
0,8.413268,8.423075,8.503907,8.501809,8.569752,8.563438
1,7.409705,7.404068,7.456415,7.438916,7.429419,7.413067
2,7.457382,7.460789,7.502171,7.491413,7.557557,7.552778
3,7.857927,7.858712,7.932425,7.919128,7.970119,7.954088
4,7.889799,7.888092,7.940813,7.916669,7.838000,7.845519


In [7]:
meta_train.head()

,catb_118_100,catb_118_200,lgb_118_100,lgb_118_200,lgb_946_100,lgb_946_200
0,7.970579,7.945658,8.007771,8.137556,8.168119,8.195903
1,6.798381,6.924421,6.880112,6.885123,6.830335,6.786534
2,7.083482,7.033327,7.054327,7.025443,6.994733,7.053505
3,6.989968,6.834804,6.963848,6.910085,6.872955,6.767249
4,6.828593,6.848638,6.794389,6.783614,6.577824,6.619822


In [8]:
print meta_train.shape,meta_test.shape

(341424, 6) (146765, 6)


In [9]:
from sklearn.metrics import mean_squared_error

In [10]:
[mean_squared_error(target,meta_train.iloc[:,a])**0.5 for a in range(6)]

[0.963153064867062,
 0.9632801366271472,
 0.9643625840377068,
 0.9644005708684165,
 0.9616761532225605,
 0.9613957124520353]

In [11]:
print mean_squared_error(target,meta_train.mean(axis=1))**0.5

0.9609527445243013


In [15]:
meta_train_100=meta_train[meta_train.columns[~meta_train.columns.str.contains('200|300')]]
meta_test_100=meta_test[meta_test.columns[~meta_test.columns.str.contains('200|300')]]

meta_train_200=meta_train[meta_train.columns[~meta_train.columns.str.contains('100|300')]]
meta_test_200=meta_test[meta_test.columns[~meta_test.columns.str.contains('100|300')]]

In [16]:
print mean_squared_error(target,meta_train_100.mean(axis=1))**0.5
print mean_squared_error(target,meta_train_200.mean(axis=1))**0.5

0.9612813133990932
0.9612972146218226


In [17]:
from sklearn.linear_model import LinearRegression

In [18]:
meta_est_100=Estimator(model=LinearRegression(),n_jobs=-1,
                          validation_scheme='KFold',
              random_state=100,n_splits=5,early_stopping_rounds=100,shuffle=True)

In [19]:
meta_oof_100=meta_est_100.fit_transform(meta_train_100.values,target.values)

0.9587301985798623
0.9602755215978305
0.9500410809147489
0.9683491276325767
0.9652900001855526


In [20]:
print meta_est_100.cv_scores, meta_est_100.avg_cv_score

[0.9587301985798623, 0.9602755215978305, 0.9500410809147489, 0.9683491276325767, 0.9652900001855526] 0.9605371857821142


In [21]:
meta_est_200=Estimator(model=LinearRegression(),n_jobs=-1,
                          validation_scheme='KFold',
              random_state=200,n_splits=5,early_stopping_rounds=100,shuffle=True)

In [22]:
meta_oof_200=meta_est_200.fit_transform(meta_train_200.values,target.values)

0.9600619076278196
0.9563922669529618
0.9602988651536486
0.961015852075746
0.9645812057682608


In [23]:
print meta_est_200.cv_scores, meta_est_200.avg_cv_score, np.std(meta_est_200.cv_scores)

[0.9600619076278196, 0.9563922669529618, 0.9602988651536486, 0.961015852075746, 0.9645812057682608] 0.9604700195156873 0.0026085896832751605


In [20]:
# joblib.dump(meta_est_100,'meta_LR_seed_100.pkl')
# joblib.dump(meta_est_200,'meta_LR_seed_200.pkl')

['meta_LR_seed_200.pkl']

In [24]:
meta_preds_100=meta_est_100.predict(meta_test_100.values)
meta_preds_200=meta_est_200.predict(meta_test_200.values)

custom_estimator_regressor.py:108: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  return np.mean(np.column_stack((est.predict(x) for est in self.fitted_models)), axis=1)


In [25]:
print mean_squared_error(target,(meta_oof_100+meta_oof_200)/2)**0.5

0.9599669793324584


In [27]:
meta_avg=(meta_preds_100+meta_preds_200)/2
sub=pd.read_csv('sample_submission.csv')
res_ids=pd.read_csv('test.csv',usecols=['reservation_id']).reservation_id
sol=pd.DataFrame({'reservation_id':res_ids,'amount_spent_per_room_night_scaled':meta_avg})
sol.to_csv('final_sub_2.csv',index=False)